## Выбираем лучшие настройки для тикеров

In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
from bot.test import TestAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import combinations


# test_configs = [
#     RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
#     RunConfig.from_repr_string('MVID+ 4/fan3:0/2 x l1 x 0.2(+x0.2)¤'),
#     RunConfig.from_repr_string('MTLR+ 2/pre4:2/2 x l1 x 1.4¤'),
#     RunConfig.from_repr_string('MTLR+ 2/fan6:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
#     RunConfig.from_repr_string('RNFT+ 2/fan6:0/2 x l1 x 0.6(+x0.2)¤'),
# ]

def generate_recombinations(string):
    result = []
    for r in range(len(string) + 1):
        for combo in combinations(string, r):
            result.append("".join(combo))
    return result


test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=pretest_period,
        pretest_type=t_config.pretest_type,

        stop_up_p=t_config.stop_up_p,

        step_size=t_config.step_size,
        step_size_shift=t_config.step_size_shift,
        step_set_orders_cnt=t_config.step_set_orders_cnt,
        mods=mods,
        # mods=t_config.mods,
    ))
    for t_config in [
        # RunConfig.from_repr_string('RNFT+ 3/fan7:0/2 x l1 x 1.2¤'),
        # RunConfig.from_repr_string('RNFT+ 3/fan7:0/2 x l1 x 1.2¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
        # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        
        # RunConfig.from_repr_string('MVID+ 5/pre3:-5/2 x l1 x 0.8¤'),
        # RunConfig.from_repr_string('MTLR+ 11/pre5:0/2 x l1 x 0.4¤ F'),
        # RunConfig.from_repr_string('VKCO+ 2/pre4:-2/2 x l1 x 2.2¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('UWGN- 4/fan3:2/2 x l1 x 2.4(+x0.2)¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('SPBE- 18/pre6:9/2 x l1 x 1.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('RNFT+ 4/pre4:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MVID+ 4/fan7:4/2 x l1 x 0.8¤ |u0.01 d0.0|'),
        
        RunConfig.from_repr_string('MVID+ 4/pre7:4/2 x l176 x 0.6¤ M'),
        RunConfig.from_repr_string('MVID+ 4/fan5:0/2 x l39 x 0.4¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MTLR+ 4/pre5:23/2 x l21 x 0.6(+x0.2)¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('MTLR+ 4/fan3:0/2 x l138 x 0.6(+x0.2)¤ |u0.01 d0.0|'),
        RunConfig.from_repr_string('RNFT+ 4/pre7:9/2 x l62 x 0.6(+x0.2)¤ |u0.01 d0.0| '),
        RunConfig.from_repr_string('RNFT+ 4/fan7:0/2 x l73 x 0.6¤ |u0.01 d0.0|'),
    ]
    for mods in ['B', 'BE', 'BR', 'BER']
    # for mods in ['', 'B', 'E', 'R']
    # for mods in ['', 'M', 'W', 'MW', 'P', 'F', 'B', 'PB', 'FMWPB', 'MWPB']
    # for mods in generate_recombinations('FMWEB')
    for max_shares in [3]
    for base_shares in [0]
    # for stop_up_p in [0]
    # for step_size_shift in [0, .2, .4]
    # for step_cnt in [2]
    for pretest_period in [t_config.pretest_period]
    # for pretest_period in range(3, 8)
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config, use_cache=True)
    return test_alg.test(
        last_test_date='2025-01-01',
        # test_days_num=30,
        test_days_num=366,
        shares_count=0,
        
        try_find_best_config=True,
    )

unique_configs = set(test_configs)

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}

    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()


print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)

last_val = ''
for item in sorted_results:
    cfg = item['config']
    exp = f"{cfg.ticker} {cfg.pretest_type} {cfg.pretest_period} {cfg.mods}"
    cur_val = f"{cfg.ticker} {cfg.pretest_type}"
    item['exp'] = exp
    if cur_val != last_val:
        last_val = cur_val
        print()
    print(item)

Запуск в 22:10
Закончено в 02:26, количество 16, длительность 2 days, 4:16:00                                      
cache 2908445 vals, used 2276441 times

{'exp': 'MVID fan 5 RB', 'profit': 4656, 'profit_p': 4.66, 'profit_p_avg': 0.01, 'config': MVID+ 4/fan5:0/2 x l1 x 0.4¤ |u0.01 d0.0| RB , 'last_conf': MVID+ 10/fan6:0/2 x l79 x 0.6¤ |u0.01 d0.0| RB , 'op': 2345}
{'exp': 'MVID fan 5 EB', 'profit': -967, 'profit_p': -0.97, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.4¤ |u0.01 d0.0| EB , 'last_conf': MVID+ 10/fan7:0/2 x l74 x 0.6¤ EB , 'op': 2078}
{'exp': 'MVID fan 5 ERB', 'profit': -1082, 'profit_p': -1.08, 'profit_p_avg': -0.0, 'config': MVID+ 4/fan5:0/2 x l1 x 0.4¤ |u0.01 d0.0| ERB , 'last_conf': MVID+ 10/fan7:0/2 x l74 x 0.6¤ ERB , 'op': 2547}
{'exp': 'MVID fan 5 B', 'profit': -2318, 'profit_p': -2.32, 'profit_p_avg': -0.01, 'config': MVID+ 4/fan5:0/2 x l1 x 0.4¤ |u0.01 d0.0| B , 'last_conf': MVID+ 10/fan4:0/2 x l53 x 0.6¤ B , 'op': 2645}

{'exp': 'MVID pre 7 B', 'pro